In [1]:
import numpy as np
import pandas as pd
import pickle
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from scipy import stats
import glob

# Load dataset

In [2]:
filenames = glob.glob("*training_data/*train.npy")
train = np.array([np.load(fname) for fname in filenames])
train = train.reshape(train.shape[1]*train.shape[0], train.shape[2])
np.random.shuffle(train)

In [3]:
Y = train[:, 0]
X = train[:, 1:]
print(Y.shape)
print(X.shape)

(12800112,)
(12800112, 9)


In [4]:
# Define training and test inds
n_train = int(Y.shape[0] * 0.5) # 80 % training
training_inds = np.arange(0, n_train)
testing_inds = np.arange(n_train, len(Y))
n_test = len(testing_inds)

print(n_train)
print(n_test)

6400056
6400056


# Add noise. Probably don't need to add noise to the date. But I should add different noise to temperature and precip

In [5]:
X_noise = np.copy(X)
add_noise = True

if add_noise:
    # Temperature (mu = 0.5 degrees)
    print()
    for j in range(0, X.shape[1] - 7):
        print(j)
        mu  = 0
        sigma = 0.5
        noise = np.random.normal(mu, sigma, X.shape[0])
        X_noise[:, j] += noise

    # Precip (mu = 5 percent)
    print()
    for j in range(2, X.shape[1] - 2):
        print(j)
        mu  = np.repeat(0, X.shape[0])
        sigma = X[:, j] / 20
        noise = np.random.normal(mu, sigma)
        X_noise[:, j] += noise


0
1

2
3
4
5
6


# Define training and testing data sets

In [6]:
# Train
X_train = X_noise[training_inds, :]
Y_train = Y[training_inds]

# Test
X_test = X[testing_inds, :]

# Train model

In [ ]:
%%time
rf_model = RandomForestRegressor(n_jobs=64, n_estimators=100)
rf_model.fit(X_train, Y_train)

# Save model

In [ ]:
# The sklearn model is held internally in the esem model
with open("rf_model.pkl","wb") as f:
    pickle.dump(rf_model, f)

# Inference

In [ ]:
%%time
Y_rf_test = rf_model.predict(X_test)

# Perform analysis

In [ ]:
x_1_to_1 = np.array([-9999, 9999])
y_1_to_1 = np.array([-9999, 9999])

In [ ]:
def calc_rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def calc_bias(predictions, targets):
    return (predictions - targets).mean()

In [ ]:
# Random Forest 
x = Y[testing_inds] * 100 # Truth (cm)
y = Y_rf_test * 100 # Prediction (cm)
lin_reg = stats.linregress(x, y)
rmse = calc_rmse(y, x)
bias = calc_bias(y, x)

# Limit
lim = np.max([np.abs(x).max(), np.abs(y).max()])

plt.figure(figsize=(9, 7))
plt.xlim([-lim, lim])
plt.ylim([-lim, lim])
plt.hexbin(x, y, extent=(-lim, lim, -lim, lim), cmap=plt.cm.Blues, bins='log', gridsize=(40, 40))
cb = plt.colorbar()
cb.set_label("Count", fontsize=22)
cb.ax.tick_params(labelsize=20) 
plt.plot(x_1_to_1, lin_reg.intercept + lin_reg.slope*x_1_to_1, 'r', label='fitted line', linewidth=2)
plt.plot(x_1_to_1, y_1_to_1, 'k--', linewidth=2)
plt.grid()
plt.xlabel("SNOWPACK: 91 day FAC change [cm]", fontsize=20)
plt.ylabel("ML: 91 day FAC change [cm]", fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
plt.title("Slope = " + str(lin_reg.slope) + \
          "\nR^2 = " + str(lin_reg.rvalue**2) + \
          ",\np = " + str(lin_reg.pvalue) + \
          ",\nRMSE = " + str(rmse) + \
          ",\nBias = " + str(bias) + \
          ",\nN = " + str(n_test) + \
          ",\nX-axis Mean Magnitude = " + str(np.abs(x).mean()),
         fontsize=20)

In [ ]:
diff = y - x
limit = np.abs(diff).max()
print(limit)

# Histogram
plt.figure(figsize=(7, 5))
plt.hist(diff, bins=75)
plt.grid()
plt.xlim([-limit, limit])
plt.xlabel("ML predicted FAC change error [cm]", fontsize=20)
plt.ylabel("Count", fontsize=20)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)